# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

## <span style='color:#2656a3'> 🗒️ The notebook is divided into the following sections:
1. Parsing new data.
2. Inserting the new data into the Feature Store.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [1]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures, calendar

# We go back into the notebooks folder
%cd notebooks

/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-
/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-/notebooks


In [2]:
# Importing the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## <span style='color:#2656a3'> 🪄 Parsing New Data
We are parsing new data setting `historical` to `False` in order to fetch real-time data. This is done for electricity prices and forecast of renewable energy. 

In order to provide real time weather measures, a weather forecast measure for the next 5 days is being fetched.

There are of course no changes to the calendar data, and therefore no new data is retrieved from it.

### <span style="color:#2656a3;">💸 Electricity Prices per day from Energinet

In [3]:
# Fetching non-historical electricity prices for area DK1
electricity_df = electricity_prices.electricity_prices(
    historical=False,
    area=["DK1"]
)

In [4]:
# Display the electricity dataframe
electricity_df

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh
0,1714608000000,2024-05-02 00:00:00,2024-05-02,0,0.10859
1,1714611600000,2024-05-02 01:00:00,2024-05-02,1,0.08160
2,1714615200000,2024-05-02 02:00:00,2024-05-02,2,0.07458
3,1714618800000,2024-05-02 03:00:00,2024-05-02,3,0.05818
4,1714622400000,2024-05-02 04:00:00,2024-05-02,4,0.07928
5,1714626000000,2024-05-02 05:00:00,2024-05-02,5,0.22920
6,1714629600000,2024-05-02 06:00:00,2024-05-02,6,0.29699
7,1714633200000,2024-05-02 07:00:00,2024-05-02,7,0.38605
8,1714636800000,2024-05-02 08:00:00,2024-05-02,8,0.43729
9,1714640400000,2024-05-02 09:00:00,2024-05-02,9,0.23457


### <span style="color:#2656a3;">☀️💨 Forecast Renewable Energy next day from Energinet

In [5]:
# # Fetching non-historical forecast of renewable energy data for area DK1
# forecast_renewable_energy_df = electricity_prices.forecast_renewable_energy(
#     historical=False,
#     area=["DK1"]
# )

In [6]:
# # Display the forecast_renewable_energy dataframe
# forecast_renewable_energy_df

### <span style="color:#2656a3;"> 🌤 Weather Measurements from Open Meteo

#### <span style="color:#2656a3;"> 🕰️ Historical Weather Measures

In [7]:
# Fetching non-historical weather data for area DK1
#historical_weather_df = weather_measures.historical_weather_measures(
#    historical=False
#)

In [8]:
# Display the first 5 rows of the dataframe
#historical_weather_df.head()

#### <span style="color:#2656a3;"> 🌈 Weather Forecast

In [9]:
# Fetching weather forecast measures for the next 5 days
weather_forecast_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [10]:
# Display the weather_forecast_df dataframe
weather_forecast_df

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1714608000000,2024-05-02 00:00:00,2024-05-02,0,14.9,66.0,0.0,0.0,0.0,0.0,13.0,21.6,41.4
1,1714611600000,2024-05-02 01:00:00,2024-05-02,1,14.2,71.0,0.0,0.0,0.0,0.0,4.0,20.5,37.1
2,1714615200000,2024-05-02 02:00:00,2024-05-02,2,13.4,73.0,0.0,0.0,0.0,2.0,70.0,21.2,36.7
3,1714618800000,2024-05-02 03:00:00,2024-05-02,3,13.2,72.0,0.1,0.1,0.0,51.0,51.0,22.3,39.2
4,1714622400000,2024-05-02 04:00:00,2024-05-02,4,12.7,73.0,0.0,0.0,0.0,2.0,78.0,21.6,38.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1715022000000,2024-05-06 19:00:00,2024-05-06,19,10.7,91.0,1.4,1.4,0.0,61.0,100.0,16.6,32.0
116,1715025600000,2024-05-06 20:00:00,2024-05-06,20,10.1,90.0,1.4,1.4,0.0,61.0,100.0,19.5,37.1
117,1715029200000,2024-05-06 21:00:00,2024-05-06,21,9.5,88.0,1.4,1.4,0.0,61.0,100.0,21.6,42.1
118,1715032800000,2024-05-06 22:00:00,2024-05-06,22,9.3,86.0,0.6,0.6,0.0,3.0,100.0,22.0,41.0


In [11]:
# Display the first 5 rows of the weather_forecast dataframe
weather_forecast_df.head(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1714608000000,2024-05-02 00:00:00,2024-05-02,0,14.9,66.0,0.0,0.0,0.0,0.0,13.0,21.6,41.4
1,1714611600000,2024-05-02 01:00:00,2024-05-02,1,14.2,71.0,0.0,0.0,0.0,0.0,4.0,20.5,37.1
2,1714615200000,2024-05-02 02:00:00,2024-05-02,2,13.4,73.0,0.0,0.0,0.0,2.0,70.0,21.2,36.7
3,1714618800000,2024-05-02 03:00:00,2024-05-02,3,13.2,72.0,0.1,0.1,0.0,51.0,51.0,22.3,39.2
4,1714622400000,2024-05-02 04:00:00,2024-05-02,4,12.7,73.0,0.0,0.0,0.0,2.0,78.0,21.6,38.9


In [12]:
weather_forecast_df.tail(5)

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
115,1715022000000,2024-05-06 19:00:00,2024-05-06,19,10.7,91.0,1.4,1.4,0.0,61.0,100.0,16.6,32.0
116,1715025600000,2024-05-06 20:00:00,2024-05-06,20,10.1,90.0,1.4,1.4,0.0,61.0,100.0,19.5,37.1
117,1715029200000,2024-05-06 21:00:00,2024-05-06,21,9.5,88.0,1.4,1.4,0.0,61.0,100.0,21.6,42.1
118,1715032800000,2024-05-06 22:00:00,2024-05-06,22,9.3,86.0,0.6,0.6,0.0,3.0,100.0,22.0,41.0
119,1715036400000,2024-05-06 23:00:00,2024-05-06,23,9.1,84.0,0.6,0.6,0.0,3.0,100.0,21.3,40.3


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access the Feature Groups and upload the new data into the Feature Groups.

In [13]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
# Retrieve the feature groups
electricity_fg = fs.get_feature_group(
    name="electricity_prices",
    version=1,
)

# forecast_renewable_energy_fg = fs.get_feature_group(
#     name="forecast_renewable_energy",
#     version=1,
# )

weather_fg = fs.get_feature_group(
    name="weather_measurements",
    version=1,
)

### <span style="color:#2656a3;"> ⬆️ Uploading new data to the Feature Store
Here we upload the new data to the retrieved Feature groups.

In [15]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df, 
                      write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17e8aa4d0>, None)

In [16]:
# # Inserting the forecast_renewable_energy_df into the feature group named forecast_renewable_energy_fg
# forecast_renewable_energy_fg.insert(forecast_renewable_energy_df, 
#                                     write_options={"wait_for_job" : False})

In [17]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(weather_forecast_df, 
                  write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/120 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17e8f7750>, None)

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 03: Traning </span>

Next we will create a feature view and training dataset. Further we will train a model and save it in model registry.